In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling
import torch

c:\Users\heman\.conda\envs\legalease310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bin c:\Users\heman\.conda\envs\legalease310\lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


In [2]:
model_name = "tiiuae/falcon-rw-1b"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

c:\Users\heman\.conda\envs\legalease310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [6]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["query_key_value", "dense"]
)

In [7]:
model = get_peft_model(model, lora_config)

In [8]:
dataset = load_dataset("json", data_files="../data/legal_qa.json", split="train")

In [9]:
def format_example(example):
    return f"### Question:\n{example['instruction']}\n\n### Answer:\n{example['output']}"

In [10]:
dataset = dataset.map(lambda x: {"text": format_example(x)})

In [11]:
tokenized = dataset.map(lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=512), batched=True)
tokenized.set_format("torch", columns=["input_ids", "attention_mask"])

In [12]:
training_args = TrainingArguments(
    output_dir="qlora-legalease",       # Saves checkpoints here
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=2,

    logging_steps=10,                            # Log loss every 10 steps
    save_strategy="steps",                       # Save checkpoint every N steps
    save_steps=100,                              # Save every 100 steps
    save_total_limit=2,                          # Keep only the 2 latest checkpoints

    evaluation_strategy="no",                    # No eval during training (can enable if needed)
    fp16=True,                                   # Mixed precision (if using CUDA)
    load_best_model_at_end=False,                # Not needed unless using eval

    report_to="none",                            # Disable logging to HuggingFace Hub
    overwrite_output_dir=True,                   # Optional: overwrite old checkpoints
)
collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized,
    data_collator=collator
)

c:\Users\heman\.conda\envs\legalease310\lib\site-packages\accelerate\accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train(resume_from_checkpoint=True)

c:\Users\heman\.conda\envs\legalease310\lib\site-packages\transformers\trainer.py:2956: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPT

{'loss': 1.8616, 'grad_norm': 1.2462979555130005, 'learning_rate': 4.573707370737074e-05, 'epoch': 0.17}


  9%|▉         | 320/3636 [13:03<32:47:26, 35.60s/it]

{'loss': 1.7543, 'grad_norm': 1.4599545001983643, 'learning_rate': 4.55995599559956e-05, 'epoch': 0.18}


  9%|▉         | 330/3636 [19:36<35:56:00, 39.13s/it]

...

 10%|▉         | 350/3636 [32:49<36:17:15, 39.76s/it]

{'loss': 1.7632, 'grad_norm': 1.4894806146621704, 'learning_rate': 4.5187018701870184e-05, 'epoch': 0.19}


 10%|▉         | 360/3636 [39:26<36:10:59, 39.76s/it]

{'loss': 1.6885, 'grad_norm': 1.5026265382766724, 'learning_rate': 4.5049504950495054e-05, 'epoch': 0.2}


 10%|█         | 365/3636 [42:45<36:09:38, 39.80s/it]

In [ ]:
training_args = TrainingArguments(
    output_dir="qlora-legalease",       # Saves checkpoints here
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=2,

    logging_steps=10,                            # Log loss every 10 steps
    save_strategy="steps",                       # Save checkpoint every N steps
    save_steps=100,                              # Save every 100 steps
    save_total_limit=2,                          # Keep only the 2 latest checkpoints

    evaluation_strategy="no",                    # No eval during training (can enable if needed)
    fp16=True,                                   # Mixed precision (if using CUDA)
    load_best_model_at_end=False,                # Not needed unless using eval

    report_to="none",                            # Disable logging to HuggingFace Hub
    overwrite_output_dir=True,                   # Optional: overwrite old checkpoints
)
collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized,
    data_collator=collator
)

# trainer.train()

c:\Users\heman\.conda\envs\legalease310\lib\site-packages\accelerate\accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
  0%|          | 10/3636 [06:19<38:04:09, 37.80s/it]

{'loss': 2.7021, 'grad_norm': 1.66172456741333, 'learning_rate': 4.9862486248624866e-05, 'epoch': 0.01}


  1%|          | 20/3636 [12:40<38:22:41, 38.21s/it]

{'loss': 2.397, 'grad_norm': 2.0429348945617676, 'learning_rate': 4.972497249724973e-05, 'epoch': 0.01}


  1%|          | 30/3636 [19:03<38:19:29, 38.26s/it]

...

  8%|▊         | 290/3636 [3:03:16<35:34:34, 38.28s/it]

{'loss': 1.7416, 'grad_norm': 1.8991179466247559, 'learning_rate': 4.601210121012102e-05, 'epoch': 0.16}


  8%|▊         | 300/3636 [3:09:37<35:04:29, 37.85s/it]

{'loss': 1.7362, 'grad_norm': 1.6861705780029297, 'learning_rate': 4.5874587458745876e-05, 'epoch': 0.17}


c:\Users\heman\.conda\envs\legalease310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
model.save_pretrained("qlora-legalease")
tokenizer.save_pretrained("qlora-legalease")